In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing

1. Видаляємо неінформативні колонки
2. Корегуємо назви колонок аби вони були коректними при роботі з layers
3. Переглядаємо датасет

In [ ]:
dataframe = pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")
dataframe = dataframe.drop(columns=["Unnamed: 32", "id"])

cns = dataframe.columns
cn_to_replace = []
for cn in cns:
    if " " in cn:
        dataframe= dataframe.rename(columns={cn: "_".join(cn.split(" "))})
        
dataframe.head()

Виділяємо половину датасету для навчання

In [ ]:
train_dataframe = dataframe.sample(frac=0.8, random_state=2021)
validate_dataframe = dataframe.drop(train_dataframe.index)

In [ ]:
train_dataframe.head()
#validate_dataframe.head()

Конвертуємо стрінгові значення до числових та виконуємо підготовку датасети

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

def create_ds(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("diagnosis")
    labels = labels.to_numpy()
    sl_layer = StringLookup(num_oov_indices=0)
    sl_layer.adapt(labels)
    labels = sl_layer(labels)
    
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size = len(dataframe))
    return ds

train_ds = create_ds(train_dataframe)
validate_ds = create_ds(validate_dataframe)

train_ds = train_ds.batch(50)
validate_ds = validate_ds.batch(50)

features = dataframe.columns[1:]

In [ ]:
features

In [ ]:
inputs = []
for feature in features:
    tmp = keras.Input(name = feature, shape = (1,), dtype = tf.float32)
    if tmp.shape[-1] is None:
        tmp = tf.expand_dims(tmp, -1)
    inputs.append(tmp)
    

f_layers = layers.concatenate(inputs)

In [ ]:
tmp = layers.Dense(17, activation="relu")(f_layers)
out = layers.Dense(1, activation="sigmoid")(tmp)
model = keras.Model(inputs, out)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [ ]:
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.fit(train_ds, epochs=20, validation_data = validate_ds)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def get_x_y_df(df):
    cdf = df.copy()
    y = cdf.pop("diagnosis")
    voc = y.unique()
    y = y.replace(["B", "M"], [1, 0])
    
    x= (cdf-cdf.min())/(cdf.max()-cdf.min())
    
    return x, y

In [ ]:
t_x, t_y = get_x_y_df(train_dataframe)
v_x, v_y = get_x_y_df(validate_dataframe)

rfc = RandomForestClassifier()

In [ ]:
rfc.fit(t_x, t_y)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(rfc.predict(v_x), v_y)